In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal, ogr
from netCDF4 import Dataset
import rasterio
import xarray as xr
import os, os.path
from shapely.geometry import Polygon, Point

%matplotlib inline
directory = '../data'

# Read SST

In [3]:
#https://www1.ncdc.noaa.gov/pub/data/cmb/ersst/v5/netcdf/
sst = xr.open_dataset(directory+"/temp/global/ersst/ersst.v5.201805.nc").to_dataframe()
sst.reset_index(inplace=True)

#better data than above
#gathered two data points for each month (Oct - May?) to get a general feel for the temp in this growth/harvest period
filelist = os.listdir(directory+"/temp/global/oisst") 
df_list = [xr.open_dataset('../data/temp/global/oisst/'+file).to_dataframe() for file in filelist]
oisst = pd.concat(df_list)
oisst.reset_index(inplace=True)
oisst['analysed_sst'] -= 273.15
oisst.head()
print(oisst.shape)
print(min(oisst.lat), max(oisst.lat),min(oisst.lon),max(oisst.lon))
quicklook = oisst[(oisst.lat >=35) & (oisst.lat <=36) & (oisst.lon >= -124) & (oisst.lon <=-122)]
quicklook.head()

,lat,lon,nv,time,lat_bnds,lon_bnds,analysed_sst,analysis_error,mask,sea_ice_fraction
0,-89.875,-179.875,0,2017-10-31,-90.00,-180.00,NaN,NaN,2.0,NaN
1,-89.875,-179.875,1,2017-10-31,-89.75,-179.75,NaN,NaN,2.0,NaN
2,-89.875,-179.625,0,2017-10-31,-90.00,-179.75,NaN,NaN,2.0,NaN
3,-89.875,-179.625,1,2017-10-31,-89.75,-179.50,NaN,NaN,2.0,NaN
4,-89.875,-179.375,0,2017-10-31,-90.00,-179.50,NaN,NaN,2.0,NaN


In [ ]:
#create mean, max, and min temps for the data
mean_temps = oisst.groupby(['lat_bnds','lon_bnds']).mean()
max_temps = oisst.groupby(['lat_bnds','lon_bnds']).max()
min_temps = oisst.groupby(['lat_bnds','lon_bnds']).min()
mean_temps.reset_index(inplace=True)
max_temps.reset_index(inplace=True)
min_temps.reset_index(inplace=True)
max_temps['max_temp'] = max_temps.analysed_sst
min_temps['min_temp'] = min_temps.analysed_sst
#remove data cols which are useless/I don't understand
mean_temps.drop(['mask','sea_ice_fraction','analysis_error','nv'], axis=1, inplace=True)
max_temps.drop(['time','mask','sea_ice_fraction','analysed_sst','lat','lon','analysis_error','nv'],\
               axis=1, inplace=True)
min_temps.drop(['time','mask','sea_ice_fraction','analysed_sst','lat','lon','analysis_error','nv'],\
               axis=1, inplace=True)

In [ ]:
#aggregate into one big df
sst_overview = mean_temps.merge(max_temps, on=["lat_bnds","lon_bnds"], how='inner')\
                         .merge(min_temps, on=["lat_bnds","lon_bnds"], how='inner')

In [ ]:
#get values for a given point. We may want to do the kirging option instead of this.
lat = 35.863
lon = -133.72
vertices = (sst_overview.iloc[np.sqrt((sst_overview.lon_bnds-lon)**2+(sst_overview.lat_bnds-lat)**2).argsort()[:3]])
vertices.reset_index(drop=True, inplace = True)
print(vertices)
#set these to find out which row contains the closest pair along the lat/long axes.
lat_pair = None
lon_pair = None
if vertices.iloc[0].lat_bnds == vertices.iloc[1].lat_bnds:
    lat_diff = abs(vertices.iloc[0].lat_bnds - vertices.iloc[2].lat_bnds)
    lon_diff = abs(vertices.iloc[0].lon_bnds - vertices.iloc[1].lon_bnds)
    lat_pair = 1
    long_pair = 2
else:
    lat_diff = abs(vertices.iloc[0].lat_bnds - vertices.iloc[1].lat_bnds)
    lon_diff = abs(vertices.iloc[0].lon_bnds - vertices.iloc[2].lon_bnds)
    lat_pair = 2
    lon_pair = 1
print("lat_diff: {}, lon_diff: {}".format(lat_diff, lon_diff))

point_lat_diff = round(abs(vertices.iloc[0].lat_bnds - lat),2)
point_lon_diff = round(abs(vertices.iloc[0].lon_bnds - lon),2)
print("point_lat_diff: {}, point_lon_diff: {}".format(point_lat_diff, point_lon_diff))
point_lat_pct = point_lat_diff/lat_diff
point_lon_pct = point_lon_diff/lon_diff
print("point_lat_pct: {}, point_lon_pct: {}".format(point_lat_pct, point_lon_pct))

lat_temp_diff = vertices.iloc[lat_pair] - vertices.iloc[0]
lon_temp_diff = vertices.iloc[lon_pair] - vertices.iloc[0]

point_lat_temp = lat_temp_diff*point_lat_pct
point_lon_temp = lon_temp_diff*point_lon_pct
#print("poin_lat_temp: {}, point_lon_temp: {}".format(point_lat_temp, point_lon_temp))

#take into account negative temperature changes
length_fraction = (point_lat_temp + point_lon_temp)/(abs(point_lat_temp)+ abs(point_lon_temp))
point_temp = vertices.iloc[0]+length_fraction*(np.sqrt(point_lat_temp**2+point_lon_temp**2))
print(point_temp)


# Read Bathymetry

In [ ]:
bath = xr.open_dataset("../../data/bathymetry/coastal_relief_maps/crm_vol7.nc").to_dataframe()
bath.reset_index(inplace=True)
print("max x: {}, min x: {}, max y :{} min y: {}".format(max(bath.x), min(bath.x), max(bath.y), min(bath.y)))
#region = bath[(bath.x >=-121.8) & (bath.x<=-121.5) & (bath.y >=34.5) & (bath.y<=34.9)]
#region.head()

In [ ]:
region = bath[(bath.x >=-124.11) & (bath.x<=-124.10) & (bath.y >=40.0) & (bath.y<=40.3)]
region.head()
bath.shape

# Read critical species habitats

In [ ]:
#read in black abalone data
#create a dictionary for reference of which species may intersect with a site
crit_species_dfs = {}
black_abalone = gpd.read_file("../../data/critical/california/black_abalone/BlackAbFinalCH10272011.shp")
crit_species_dfs['black_abalone'] = black_abalone
# doesn't return any new info afaict
#babalshz = gpd.read_file("../../data/critical/california/black_abalone/BlackAbFinalCH10272011.shz")
#babalshz.head()
#site_intersections = sum(black_abalone.intersects(Polygon([[-123, 39],[-123,32],[-120,39],[-120,32]])))

leatherback = gpd.read_file("../../data/critical/california/leatherback/Final_LeatherbackCH.shp")
crit_species_dfs['leatherback'] = leatherback

stellar_sealions = gpd.read_file("../../data/critical/california/stellar_sealions/StellerSLionCritHab_OR_CA.shp")
crit_species_dfs['stellar_sealions'] = stellar_sealions

#green sturgeon had many shp files, most of which were irrelevant for our immediate purposes.
#I concatenated to relevant ones
#not relevant
#GreenSturgeonCHBypassAreas.shp (interior california)
#GreenSturgeonCHDepartmentofDefense.shp (not in ca)
#GreenSturgeonCHHeadOfTide.shp (interior)
#GreenSturgeonCHMarshAreas.shp (interior)
#GreenSturgeonCHStreams.shp (streams)
#GreenSturgeonCHTribalExclusions.shp

#relevant
#GreenSturgeonCHEstuaries.shp
#GreenSturgeonCHMarineCoastalZones.shp

green_sturgeon = gpd.read_file\
("../../data/critical/california/green_sturgeon/GreenSturgeonFinalCH/GreenSturgeonCHEstuaries.shp")
green_sturgeon = green_sturgeon.append(gpd.read_file\
("../../data/critical/california/green_sturgeon/GreenSturgeonFinalCH/GreenSturgeonCHMarineCoastalZones.shp"))
crit_species_dfs['green_sturgeon'] = green_sturgeon